Importeringar för Notebooken

In [1]:
import matplotlib.pyplot as plt
import pandas as pd



Beskrivande analys

Räkna ut medel, median, min och max för: age, weight, height, systolic_bp, cholesterol.
Skapa minst 3 olika grafer (t.ex. histogram över blodtryck, boxplot över vikt per kön, stapeldiagram över andelen rökare).

Simulering kopplad till caset (använd numpy.random + seed)

Beräkna andelen personer i datasetet som har sjukdomen.
Använd numpy för att simulera 1000 slumpade personer med samma sannolikhet för sjukdom.
Jämför den simulerade andelen med den verkliga andelen i datasetet.

Konfidensintervall

Beräkna ett konfidensintervall för medelvärdet av systolic_bp (t.ex. med normalapproximation eller bootstrap, som vi gått igenom på kursen).

Hypotesprövning

Testa hypotesen: ”Rökare har högre medel-blodtryck än icke-rökare.”
Använd en av metoderna vi gått igenom (t.ex. bootstrap eller enkel t-test-funktion).
Förklara kort resultatet i en markdown-ruta.

Förbereder data

In [2]:
#Read

#do I even need this whole section since we are writing in the same page? 
REQUIRED = ["id","age","sex","height","weight","systolic_bp","cholesterol","smoker","disease"]

def load_data(path: str) -> pd.DataFrame:
    """
    Läser CSV 
    """
    df = pd.read_csv(path)
    missing = [c for c in REQUIRED if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    return df

df = load_data("C:/Users/Mauro/Desktop/Del 1/Health-study-project/health_study_dataset.csv")


def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensures numerical columns are numerical category
    """
    out = df.copy()
    for c in {"id","age","height","weight","systolic_bp","cholesterol", "disease"}:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    return out


df = df.dropna(subset=["id","age","sex","height","weight","systolic_bp","cholesterol","smoker","disease"]).copy()
#df = df[df["id","age","sex","height","weight","systolic_bp","cholesterol","smoker","disease"]>0].copy()
#behövs inte

df.info()

#Clean

#Visualize

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           800 non-null    int64  
 1   age          800 non-null    int64  
 2   sex          800 non-null    object 
 3   height       800 non-null    float64
 4   weight       800 non-null    float64
 5   systolic_bp  800 non-null    float64
 6   cholesterol  800 non-null    float64
 7   smoker       800 non-null    object 
 8   disease      800 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 56.4+ KB


In [ ]:
#Metrics
#Räkna ut medel, median, min och max för: age, weight, height, systolic_bp, cholesterol.

columns = ["age", "weight", "height", "systolic_bp", "cholesterol"]
averages = df[columns].mean()
medians = df[columns].median()
min = df[columns].min()
max = df[columns].max()


summary = pd.DataFrame({
    "Average": averages,
    "Median": medians,
    "Minimum": min,
    "Maximum": max})

print(summary)


                Average  Median  Minimum  Maximum
age           49.426250   50.00     18.0    90.00
weight        73.413000   73.20     33.7   114.40
height       171.849250  171.35    144.4   200.40
systolic_bp  149.178625  149.40    106.8   185.90
cholesterol    4.929150    4.97      2.5     7.88


In [ ]:
#Skapa minst 3 olika grafer (t.ex. histogram över blodtryck, boxplot över vikt per kön, stapeldiagram över andelen rökare)
